In [ ]:
#file navigation tools
from glob import glob
import os
#image analysis library
from skimage import io
#jupyter notebook img display
import stackview
#The fundamental package for scientific computing with Python
import numpy as np
#python image viewer 
import napari
#excel for python
import pandas as pd

Download example data for the segmentation tutorial from https://zenodo.org/record/7004556. You can also download all the data by executing the code block below.

In [ ]:
#%cd D:\path\where\you\want\to\save\data
#!zenodo_get -d 10.5281/zenodo.7004556 

In [ ]:
#change path to location of saved files
path='location\of\data\Experiment_01*'
files=glob(path)
files

Load the first image into memory.

In [ ]:
img= io.imread(files[0],plugin='pil')

Inspect the dimensions of the loaded image.

In [ ]:
img.shape

Reshape image to create 4d image stack

In [ ]:
img=np.expand_dims(img,1)
img=np.reshape(img,(4,41,2304,2304))
img.shape

Inspect the different fluorescent channels and Ztiles of the loaded img

In [ ]:
c=0 # change this between 0 and 2 to switch channel 
stackview.slice(img[c,...])

Describe which layers of the image are in focus.

In [ ]:
focus=[23,40]

Perform a max projection on the different channels of the image.

In [ ]:
zproject=np.amax(img[:,focus[0]:focus[1],...],axis=1,keepdims=False)
zproject.shape

Inspect the Zprojected image. Which channel number belongs to which fluorescent dye (Dapi,CY3, CY5)?

In [ ]:
stackview.slice(zproject)

Adjust the channel order accordingly. Additionally choose the color these channels should be displayed in.

In [ ]:
channels=['CY3','CY5','DAPI','CY3.5']
colors=['green','magenta','blue','cyan']

Load the DIC img.

In [ ]:
DIC= io.imread(files[1])
DIC.shape

In [ ]:
stackview.slice(DIC)

Run Napari and open the napari-buds plugin as indicated in the Tutorial_Napari_Buds.pdf.

In [ ]:
viewer= napari.Viewer()

viewer.add_image(DIC, name='DIC', opacity=0.50)
for i in range(zproject.shape[0]):
    viewer.add_image(zproject[i,...],name=channels[i],colormap=colors[i],opacity=0.30)

Excute this cell after performing and correcting cell and bud segmentation in Napari buds to save the resulting cell and bud masks.

In [ ]:
path=r"D:\save\to\your\folder"
filename="fov_000.tif"

# make nuclei directory if does not exist
if not os.path.isdir(os.path.join(path,'nuclei')):
    os.makedirs(os.path.join(path,'nuclei'))

# make cell mask if doesn't exist
if not os.path.isdir(os.path.join(path,'cells')):
    os.makedirs(os.path.join(path,'cells'))

# make buds mask directory
if not os.path.isdir(os.path.join(path,'buds')):
    os.makedirs(os.path.join(path,'buds'))

io.imsave(os.path.join(path,'nuclei',filename),viewer.layers['seeds'].data)
io.imsave(os.path.join(path,'cells',filename),viewer.layers['cell mask'].data)
io.imsave(os.path.join(path,'buds',filename),viewer.layers['buds'].data)